# Deploy Auto-AI model image on SAP AI CORE

The following cell installs the necessary Python libraries.

In [2]:
!pip install ai-core-sdk
!pip install python-dotenv

Import the following libraries:

In [1]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
import os
import requests

Below cell loads environment variables from the `.env` file located within the same folder

In [2]:
# Reads .env file from the same folder
from dotenv import load_dotenv
load_dotenv()


True

### Create SAP AI Core client
Create SAP AI Core client by authenticating using the credentials stored in `.env` file

In [3]:
ai_core_client = AICoreV2Client(
# `AI_API_URL`
   base_url = os.getenv("BASEURL"),
# `URL`
   auth_url =  os.getenv("AUTHURL"),
# `clientid`
  client_id = os.getenv("CLIENTID"),
# `clientsecret`
  client_secret = os.getenv("CLIENTSECRET")
)

### List Resource Groups
List all the resource groups in the sap ai core account 

In [4]:
response = ai_core_client.resource_groups.query()
for app in response.resources:
    print(app.__dict__)

{'resource_group_id': 'vikrambhat', 'labels': [], 'status': 'PROVISIONED', 'created_at': datetime.datetime(2023, 7, 4, 16, 5, 9)}
{'resource_group_id': 'default', 'labels': [], 'status': 'PROVISIONED', 'created_at': datetime.datetime(2023, 6, 30, 9, 6, 9)}


## Add docker Registry

Create a docker registry in the sap ai core account. This is required to pull images from docker container

In [ ]:
ai_core_client.

In [5]:
# read docker registry password stored in the .env file 
dockerregistrypassword=os.getenv("DOCKERREGISTRY")

In [6]:
response = ai_core_client.docker_registry_secrets.create(
    name = "vbrepo",
    data = {
        ".dockerconfigjson": "{\"auths\":{\"https://index.docker.io\":{\"username\":\"vikrambhat2\",\"password\":\""+dockerregistrypassword+"\"}}}"
    }
)

print(response.__dict__)


{'message': 'Secret exists'}


## Add Github Repo

In [7]:
# read github access token stored in the .env file 
github_access_token=os.getenv("GITHUBTOKEN")

In [ ]:
ai_core_client.repositories.create(
    name = "vikram-aicore-github-pipelines",
    url = "https://github.com/vikrambhat2/sapaicore_mlexp",
    username = "vikrambhat2",
    password = github_access_token
)

## Check github onboarding status

In [8]:
# check on-boarding status
response = ai_core_client.repositories.query()
#
for repository in response.resources:
    print('Name:', repository.name)
    print('URL:', repository.url)
    print('Status:', repository.status)

Name: vikram-aicore-github-pipelines
URL: https://github.com/vikrambhat2/sapaicore_mlexp
Status: RepositoryStatus.COMPLETED


### Specify the folder containing yaml file

In [9]:
yaml_path="AUTOAIEXEC"

### Read execution file(yaml) and create an application

In [10]:
## Code to delete existing application
res=ai_core_client.applications.delete(application_name="aicore-app-autoai")
res.message

'Application has been successfully deleted.'

In [11]:
github_app_name="aicore-app-autoai"
ai_core_client.applications.create(
    application_name = github_app_name,
    repository_url = "https://github.com/vikrambhat2/sapaicore_mlexp",
    path = yaml_path,
    revision = "HEAD"
)


In [12]:
response = ai_core_client.applications.query()

for app in response.resources:
    print(app.__dict__)

{'path': 'AUTOAIEXEC', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-autoai'}
{'path': 'DemandResponse', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-scikit'}
{'path': 'DemandResponseExec', 'revision': 'HEAD', 'repository_url': 'https://github.com/vikrambhat2/sapaicore_mlexp', 'application_name': 'aicore-app-vb'}


### Check if execution file sync is successful

In [13]:
response = ai_core_client.applications.get_status(application_name=github_app_name)


for workflow_sync_status in response.sync_ressources_status:
    print(workflow_sync_status.__dict__)
print(response.__dict__)
print('*'*80)
print(response.sync_ressources_status[0].__dict__)


{'name': 'auto-pipeline', 'kind': 'ServingTemplate', 'status': 'Synced', 'message': 'servingtemplate.ai.sap.com/auto-pipeline created'}
{'health_status': 'Healthy', 'sync_status': 'Synced', 'message': 'successfully synced (all tasks run)', 'source': <ai_core_sdk.models.application_source.ApplicationSource object at 0x7fb4b87d0310>, 'sync_finished_at': '2023-09-21T10:50:42Z', 'sync_started_at': '2023-09-21T10:50:40Z', 'reconciled_at': '2023-09-21T10:50:43Z', 'sync_ressources_status': [<ai_core_sdk.models.application_resource_sync_status.ApplicationResourceSyncStatus object at 0x7fb4b87d5700>]}
********************************************************************************
{'name': 'auto-pipeline', 'kind': 'ServingTemplate', 'status': 'Synced', 'message': 'servingtemplate.ai.sap.com/auto-pipeline created'}


## Create Configuration

In [14]:
exec_id="auto-pipeline"
scen_id="demandresponse-autoai"

In [15]:
from ai_core_sdk.models import InputArtifactBinding
from ai_api_client_sdk.models.parameter_binding import ParameterBinding

response = ai_core_client.configuration.create(
    name = "demandresponse-autoai-withaicore",
    resource_group = "vikrambhat",
    scenario_id = scen_id, # value from workflow
    executable_id = exec_id,
#        input_artifact_bindings = [
#        # list
#        InputArtifactBinding(key="demandresponsemodel", artifact_id =artifact_id), # Change artifact id
#    ]

)

print(response.__dict__)

{'id': '155c3ec9-af89-4aad-9fef-cbb2e01b52eb', 'message': 'Configuration created'}


In [16]:
conf_id=response.__dict__['id']

In [17]:
# Lists all configurations
response = ai_core_client.configuration.query(
    resource_group = "vikrambhat"
)

for configuration in response.resources:
    print(configuration.__dict__)
    


{'id': '155c3ec9-af89-4aad-9fef-cbb2e01b52eb', 'name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'executable_id': 'auto-pipeline', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2023, 9, 21, 10, 51, 6), 'scenario': None}
{'id': 'baa71b89-6516-4ba0-ac15-52724fa763e8', 'name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'executable_id': 'auto-pipeline', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2023, 9, 1, 10, 31, 52), 'scenario': None}
{'id': 'c755c5fb-cd81-4178-9859-b21ba71255cf', 'name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'executable_id': 'auto-pipeline', 'parameter_bindings': [], 'input_artifact_bindings': [], 'created_at': datetime.datetime(2023, 8, 30, 15, 52, 5), 'scenario': None}
{'id': 'd010e4fb-3bc8-4cdf-a5a5-0c6cdcb7bfad', 'name': 'demandresponse-autoai-withaicore', 'scenario_id': 'de

## Create Deployment

In [18]:
response = ai_core_client.deployment.create(
    resource_group = 'vikrambhat',
    configuration_id = conf_id # change this
)
print(response.__dict__)
deploymentid=response.__dict__['id']

{'id': 'd16ea8a28e3116cc', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


## Check deployment status

In [25]:
response = ai_core_client.deployment.get(
    resource_group = 'vikrambhat',
    deployment_id = deploymentid # Change this
)

print("Status: ", response.status)
print('*'*80)
print(response.__dict__)


Status:  Status.RUNNING
********************************************************************************
{'id': 'd16ea8a28e3116cc', 'configuration_id': '155c3ec9-af89-4aad-9fef-cbb2e01b52eb', 'configuration_name': 'demandresponse-autoai-withaicore', 'scenario_id': 'demandresponse-autoai', 'status': <Status.RUNNING: 'RUNNING'>, 'target_status': <TargetStatus.RUNNING: 'RUNNING'>, 'created_at': datetime.datetime(2023, 9, 21, 10, 51, 41), 'modified_at': datetime.datetime(2023, 9, 21, 10, 53, 52), 'status_message': None, 'status_details': None, 'submission_time': datetime.datetime(2023, 9, 21, 10, 52, 59), 'start_time': datetime.datetime(2023, 9, 21, 10, 53, 47), 'completion_time': None, 'deployment_url': 'https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d16ea8a28e3116cc', 'last_operation': <Operation.CREATE: 'CREATE'>, 'latest_running_configuration_id': '155c3ec9-af89-4aad-9fef-cbb2e01b52eb', 'ttl': None}


In [24]:
from ai_core_sdk.models import TargetStatus
response=ai_core_client.deployment.query_logs(resource_group = 'vikrambhat',
    deployment_id = deploymentid)
for log in response.data.result:
    print(log.msg)
    print("---")

## Make prediction

In [26]:
deployment_url = "https://api.ai.prod.us-east-1.aws.ml.hana.ondemand.com/v2/inference/deployments/d16ea8a28e3116cc"


In [27]:


# URL


endpoint = f"{deployment_url}/v2/greet" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "vikrambhat"}
response = requests.get(endpoint, headers=headers)

print(response.text)

Demand Response Model is loaded.


In [28]:

# Preparing the input for inference
test_input={'input_data': [{'fields': ['CUSTOMER_ID','CSTFNM','CSTLNM','CSTPH1','CSTML','AGE','CITY','MARITAL_STATUS','GENDER','EDUCATION','EMPLOYMENT','TENURE','SEGMENT','HOME_SIZE','ENERGY_USAGE_PER_MONTH','ENERGY_EFFICIENCY','IS_REGISTERED_FOR_ALERTS','OWNS_HOME','COMPLAINTS','EST_INCOME','CLTV','HAS_THERMOSTAT','HAS_HOME_AUTOMATION','PHOTOVOLTAIC_ZONING','WIND_ZONING','SMART_METER_COMMENTS','IS_CAR_OWNER','HAS_EV','HAS_PHOTOVOLTAIC','HAS_WIND','EBILL','IN_WARRANTY','STD_YRLY_USAGE','MISSED_PAYMENT','YEARLY_USAGE_PREDICTED'],
   'values': [[1,'Leonor','Cummerata','781-757-3537','Leonor.Cummerata@nola.biz',44,'Mountain View','S','male','Associate degree','Employed full-time',13,'Green Advocate',1980,5080,0.39,True,True,False,47870,352.0,True,False,True,False,'Negative',True,True,True,False,False,True,56049,False,59972.4]]}]}

endpoint = f"{deployment_url}/v2/predict" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "vikrambhat",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=test_input)
print(response)
print( response.json())

<Response [200]>
{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0.0, [0.6210089111178061, 0.37899108888219385]]]}]}


### Prediction and Logging to Openscale 
#### Method 1. Without Watson OpenScale Inputs

In [43]:
# sample 10 records
json_input={'input_data': [{'fields': ['CUSTOMER_ID','CSTFNM','CSTLNM','CSTPH1','CSTML','AGE','CITY','MARITAL_STATUS','GENDER','EDUCATION','EMPLOYMENT','TENURE','SEGMENT','HOME_SIZE','ENERGY_USAGE_PER_MONTH','ENERGY_EFFICIENCY','IS_REGISTERED_FOR_ALERTS','OWNS_HOME','COMPLAINTS','EST_INCOME','CLTV','HAS_THERMOSTAT','HAS_HOME_AUTOMATION','PHOTOVOLTAIC_ZONING','WIND_ZONING','SMART_METER_COMMENTS','IS_CAR_OWNER','HAS_EV','HAS_PHOTOVOLTAIC','HAS_WIND','EBILL','IN_WARRANTY','STD_YRLY_USAGE','MISSED_PAYMENT','YEARLY_USAGE_PREDICTED'], 'values': [[64988, 'Kianna', 'Torphy', '319-548-0030', 'Kianna.Torphy@precious.me', 28, 'Palo Alto', 'U', 'male', 'Some college', 'Retired', 4, 'Budget Payment Plan Members', 3330, 4500, 0.74, True, True, True, 78000, 370.0, True, True, True, True, 'Negative', True, False, True, True, True, True, 32345, False, 1215.5], [64989, 'Jess', 'Quitzon', '504-864-5217', 'Jess.Quitzon@eddie.us', 57, 'Santa Clara', 'U', 'male', "Bachelor's degree or more", 'Stay at home parent', 72, 'Budget Payment Plan Members', 1420, 2330, 0.609, True, True, False, 8073, 324.0, True, False, True, False, 'Positive', False, True, False, False, True, True, 32345, False, 7179.6], [64990, 'Christiana', 'Cummerata', '919-398-7367', 'Christiana.Cummerata@bryon.tv', 69, 'Palo Alto', 'U', 'male', 'High school graduate', 'Retired', 44, 'Budget Payment Plan Members', 4740, 14600, 0.325, False, False, False, 7737, 191.0, True, True, True, False, 'Negative', True, False, False, False, False, True, 32345, False, 6834.1], [64991, 'Damien', 'Koss', '347-730-5387', 'Damien.Koss@andreanne.biz', 57, 'Santa Clara', 'M', 'male', "Bachelor's degree or more", 'Employed full-time, Student', 50, 'Budget Payment Plan Members', 3540, 8160, 0.434, True, True, False, 28313, 128.0, False, True, False, False, 'Neutral', False, True, False, False, False, True, 32345, False, 10029.8], [64992, 'Aiyana', 'Durgan', '832-300-2448', 'Aiyana.Durgan@gavin.info', 37, 'Sunnyvale', 'S', 'female', "Bachelor's degree or more", 'Employed full-time', 61, 'High Flyers', 4240, 9120, 0.465, False, True, False, 83287, 111.0, True, False, True, True, 'Negative', False, False, False, True, True, False, 32345, True, 6687.5], [64994, 'Lottie', 'Pouros', '651-455-9857', 'Lottie.Pouros@javon.com', 64, 'Mountain View', 'S', 'female', "Bachelor's degree or more", 'Student', 26, 'Budget Payment Plan Members', 3300, 8130, 0.406, False, True, False, 4316, 128.0, False, False, True, False, 'Positive', False, False, True, False, True, False, 32345, False, 5796.6], [64995, 'Roger', 'Hagenes', '978-761-4394', 'Roger.Hagenes@tamara.tv', 53, 'Sunnyvale', 'U', 'female', 'High school graduate', 'Employed full-time', 15, 'High Flyers', 2500, 7050, 0.355, True, False, False, 20078, 111.0, True, False, False, True, 'Neutral', False, False, False, False, True, False, 32345, False, 2889.3], [64997, 'Allene', 'Bergnaum', '405-858-3428', 'Allene.Bergnaum@christopher.info', 43, 'Santa Clara', 'S', 'female', 'Some college', 'Employed full-time, Student', 7, 'High Flyers', 5410, 8120, 0.667, False, True, False, 83287, 78.8, True, False, False, False, 'Negative', False, True, False, False, False, True, 32345, False, 7369.6], [64998, 'Bryana', 'Witting', '412-423-6899', 'Bryana.Witting@mazie.biz', 62, 'Santa Clara', 'U', 'male', "Bachelor's degree or more", 'Not currently employed', 67, 'Green Advocate', 4320, 6470, 0.667, True, True, False, 73338, 127.0, True, False, False, False, 'Neutral', True, False, False, False, True, False, 32345, False, 5826.6], [64999, 'Gerson', 'Beatty', '858-564-7970', 'Gerson.Beatty@bernhard.tv', 73, 'Palo Alto', 'M', 'male', 'Some college', 'Student', 70, 'High Flyers', 2760, 3450, 0.8, True, False, False, 9114, 158.0, False, False, False, False, 'Neutral', True, False, False, False, False, False, 32345, False, 1058.1]]}]}



endpoint = f"{deployment_url}/v2/predict_and_log" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "vikrambhat",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=json_input)
print(response)
print( response.json())

<Response [200]>
{'model_prediction': {'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0.0, [0.6210089111178061, 0.37899108888219385]], [1.0, [0.31457225323657423, 0.6854277467634258]]]}]}, 'response': 'Payload Logging and Feedback logging Successful'}


#### Method 2. With Watson OpenScale Inputs

In [43]:
json_input={'input_data': [{'fields': ['CUSTOMER_ID','CSTFNM','CSTLNM','CSTPH1','CSTML','AGE','CITY','MARITAL_STATUS','GENDER','EDUCATION','EMPLOYMENT','TENURE','SEGMENT','HOME_SIZE','ENERGY_USAGE_PER_MONTH','ENERGY_EFFICIENCY','IS_REGISTERED_FOR_ALERTS','OWNS_HOME','COMPLAINTS','EST_INCOME','CLTV','HAS_THERMOSTAT','HAS_HOME_AUTOMATION','PHOTOVOLTAIC_ZONING','WIND_ZONING','SMART_METER_COMMENTS','IS_CAR_OWNER','HAS_EV','HAS_PHOTOVOLTAIC','HAS_WIND','EBILL','IN_WARRANTY','STD_YRLY_USAGE','MISSED_PAYMENT','YEARLY_USAGE_PREDICTED'],
   'values': [[1,'Leonor','Cummerata','781-757-3537','Leonor.Cummerata@nola.biz',44,'Mountain View','S','male','Associate degree','Employed full-time',13,'Green Advocate',1980,5080,0.39,True,True,False,47870,352.0,True,False,True,False,'Negative',True,True,True,False,False,True,56049,False,59972.4],
             [106,'Alberta','Hackett','520-430-9350','Alberta.Hackett@albina.info',52,'Santa Clara','M','female','High school graduate','Employed full-time',14,'Budget Payment Plan Members',2120,3340,0.635,True,True,False,65619,141.0,True,True,False,True,'Positive',True,True,False,False,True,True,28395,False,28395.0]]}],
           'service_credentials':{'ibmcloudapikey':'KDuYn_OWhzQ4O5zqoo9mHEWv-ouFMiATJcUGJAUJFOZo','SERVICE_INSTANCE_ID':'b78d9fcd-74b7-4b17-a6bf-1983299bafc5','subscription_id':'9d44e345-0e39-41c7-8645-3dd60c9fd003'}
           }

endpoint = f"{deployment_url}/v2/predict_and_logging" # endpoint implemented in serving engine
headers = {"Authorization": ai_core_client.rest_client.get_token(),
           'ai-resource-group': "vikrambhat",
           "Content-Type": "application/json"}
response = requests.post(endpoint, headers=headers, json=json_input)

print(response)
print( response.json())

<Response [200]>
{'model_prediction': {'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0.0, [0.6210089111178061, 0.37899108888219385]], [1.0, [0.31457225323657423, 0.6854277467634258]]]}]}, 'response': 'Payload Logging and Feedback logging Successful'}
